With Reference to :
https://towardsdatascience.com/facial-recognition-using-deep-learning-a74e9059a150
and coursera Andrew's CNN assignment

In [1]:
import dlib
import scipy.misc
import numpy as np
import os# Get Face Detector from dlib
# This allows us to detect faces in images
face_detector = dlib.get_frontal_face_detector()# Get Pose Predictor from dlib
# This allows us to detect landmark points in faces and understand the pose/angle of the face
shape_predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')# Get the face recognition model
# This is what gives us the face encodings (numbers that identify the face of a particular person)
face_recognition_model = dlib.face_recognition_model_v1('./dlib_face_recognition_resnet_model_v1.dat')# This is the tolerance for face comparisons
# The lower the number - the stricter the comparison
# To avoid false matches, use lower value
# To avoid false negatives (i.e. faces of the same person doesn't match), use higher value
# 0.5-0.6 works well
TOLERANCE = 0.6

In [2]:
import matplotlib.pyplot as plt

In [34]:
import scipy.misc
import cv2
# This function will take an image and return its face encodings using the neural network
def get_face_encodings(path_to_image):
    # Load image using scipy
#     from PIL import Image
#     image = cv2.imread(path_to_image)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #     image = scipy.misc.imread(path_to_image)  # Detect faces using the face detector
    import matplotlib.pyplot
    image = matplotlib.pyplot.imread(path_to_image)
    detected_faces = face_detector(image, 1)    # Get pose/landmarks of those faces
    # Will be used as an input to the function that computes face encodings
    # This allows the neural network to be able to produce similar numbers for faces of the same people, regardless of camera angle and/or face positioning in the image
    shapes_faces = [shape_predictor(image, face) for face in detected_faces]    # For every face detected, compute the face encodings
    return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

In [35]:
database = {}

In [36]:
def add_user_img_path(label, img_path):
    encodings = get_face_encodings(img_path)
    database[label] = encodings

In [37]:
name = "buckley"
img_path = "./siamese dataset/A.J._Buckley/0.jpg"
add_user_img_path(name, img_path)

In [38]:
name = "amir"
img_path = "./siamese dataset/Aamir_Khan/2.jpg"
add_user_img_path( name, img_path)

In [40]:
name = "staton"
img_path = "./siamese dataset/Aaron_Staton/0.jpg"
add_user_img_path( name, img_path)

In [41]:
name = "Deyn"
img_path = "./siamese dataset/Agyness_Deyn/2.jpg"
add_user_img_path(name, img_path)

In [43]:

name = "Devgn"
img_path = "./siamese dataset/Ajay_Devgn/20.jpg"
add_user_img_path( name, img_path)

In [44]:
name = "Hodge"
img_path = "./siamese dataset/Aldis_Hodge/21.jpg"
add_user_img_path( name, img_path)

In [45]:
name = "Kumar"
img_path = "./siamese dataset/Akshay_Kumar/18.jpg"
add_user_img_path( name, img_path)

In [47]:
name = "Fast"
img_path = "./siamese dataset/Alexia_Fast/57.jpg"
add_user_img_path( name, img_path)

In [49]:
name = "Daddario"
img_path = "./siamese dataset/Alexandra_Daddario/57.jpg"
add_user_img_path( name, img_path)


In [6]:
def ini_user_database():
    # check for existing database
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        # make a new one
        # we use a dict for keeping track of mapping of each person with his/her face encoding
        user_db = {}
        # create the directory for saving the db pickle file
        os.makedirs('database')
        with open('database/user_dict.pickle', 'wb') as handle:
            pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)   
    return user_db


In [50]:
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

import imutils
from imutils import face_utils

def get_face(image):
    

  rects = detector(image,1)
  if len(rects) == 1:
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for face region , then
        # convert the facial landmark(x,y) coordinates to np array
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)
        
        # convert dlib's rectangle to openCV style bounding box 
        # i.e (x,y,w,h) then draw face bounding box
        (x,y,w,h) = face_utils.rect_to_bb(rect)
        if x>1 and y>1:
          face = image[y-15:y+h+15,x-15:x+w+15]
          face = cv2.resize(face,(int(96),int(96)))
          face = np.asarray(face)
          plt.imshow(face)
          face = face.reshape(1,3,96,96)
          # print(x,y,w,h)
      # image = cv2.resize(image,(int(128),int(128)))
          # print(type(face))
          return face
  else:
    return ''  

In [51]:
def img_to_encoding(X,network):
    img = cv2.imread(X)
    img = get_face(img)
#     img = cv2.resize(img, (int(96),int(96)))
#     img =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     img = cv2.resize(image, (int(96),int(96)))

#     img = img.reshape(1,3,96,96)
    m = img.shape[0]
    nbevaluation = int(m*(m-1)/2)
    probs = np.zeros((nbevaluation))
    y = np.zeros((nbevaluation))
    
    #Compute all embeddings for all pics with current network
    embeddings = network.predict(img)
    return embeddings

In [64]:
def find_face(image_path,threshold = 0.6):
    # find the face encodings for the input image
#     encoding = img_to_encoding(image_path, model)
    encoding = get_face_encodings(image_path)    
    
    min_dist = 99999
    # loop over all the recorded encodings in database 
    for name in database:
        # find the similarity between the input encodings and claimed person's encodings using L2 norm
        dist = np.linalg.norm(np.subtract(database[name], encoding) )
        print("name : ", name)
        print("distance ", dist)
        # check if minimum distance or not
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        print("User not in the database.")
        identity = 'Unknown Person'
    else:
        print ("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
        
    return min_dist, identity

In [73]:

find_face(image_path = "./siamese dataset/Ajay_Devgn/2.jpg")

name :  buckley
distance  0.9231899520706222
name :  amir
distance  0.6171786779369691
name :  staton
distance  0.9038289612913356
name :  Deyn
distance  0.9963270891110149
name :  Devgn
distance  0.3675174807497336
name :  Hodge
distance  0.8871550005784846
name :  Kumar
distance  0.7678247950491384
name :  Fast
distance  0.9446914045960852
name :  Daddario
distance  0.9911690016436425
Hi! Devgn, L2 distance: 0.3675174807497336


(0.3675174807497336, 'Devgn')

In [74]:
find_face(image_path = "./siamese dataset/Agyness_Deyn/19.jpg")

name :  buckley
distance  0.7688638451152058
name :  amir
distance  0.8518865416483034
name :  staton
distance  0.8938867837970622
name :  Deyn
distance  0.30571698154545834
name :  Devgn
distance  0.9748616460853747
name :  Hodge
distance  1.0945250559177562
name :  Kumar
distance  0.8092348213310246
name :  Fast
distance  0.8432310605802651
name :  Daddario
distance  0.8718128224229162
Hi! Deyn, L2 distance: 0.30571698154545834


(0.30571698154545834, 'Deyn')

In [75]:
find_face(image_path = "./siamese dataset/Aldis_Hodge/28.jpg")

name :  buckley
distance  0.957624882917737
name :  amir
distance  0.9413696593280567
name :  staton
distance  0.9590817559534535
name :  Deyn
distance  1.0818515569973681
name :  Devgn
distance  0.749654571058605
name :  Hodge
distance  0.27392696470312194
name :  Kumar
distance  0.965175565814752
name :  Fast
distance  1.0463017424335361
name :  Daddario
distance  1.0514898727311277
Hi! Hodge, L2 distance: 0.27392696470312194


(0.27392696470312194, 'Hodge')

In [76]:
find_face(image_path = "./siamese dataset/Akshay_Kumar/44.jpg")

name :  buckley
distance  0.8154795018897328
name :  amir
distance  0.7258255747435582
name :  staton
distance  0.9104891527828546
name :  Deyn
distance  0.8500959922074564
name :  Devgn
distance  0.7534095941712102
name :  Hodge
distance  0.8904417070580244
name :  Kumar
distance  0.5014233887428697
name :  Fast
distance  0.9202106911060706
name :  Daddario
distance  1.0004876666971745
Hi! Kumar, L2 distance: 0.5014233887428697


(0.5014233887428697, 'Kumar')

In [77]:
find_face(image_path = "./siamese dataset/Alexia_Fast/17.jpg")

name :  buckley
distance  0.9327600388499849
name :  amir
distance  0.9867280738866282
name :  staton
distance  0.8300859241503626
name :  Deyn
distance  0.9154851717490674
name :  Devgn
distance  1.0065891988981432
name :  Hodge
distance  1.111129497031955
name :  Kumar
distance  0.9711824594471692
name :  Fast
distance  0.4507520346859727
name :  Daddario
distance  0.871149132771125
Hi! Fast, L2 distance: 0.4507520346859727


(0.4507520346859727, 'Fast')

In [78]:
find_face(image_path = "./siamese dataset/Alexandra_Daddario/0.jpg")

name :  buckley
distance  0.9062719686840376
name :  amir
distance  0.9194762018726679
name :  staton
distance  0.8834574824997554
name :  Deyn
distance  0.8003598374684854
name :  Devgn
distance  1.0291756034279715
name :  Hodge
distance  1.0831327946020117
name :  Kumar
distance  1.0006629672789935
name :  Fast
distance  0.7611308892161551
name :  Daddario
distance  0.3275074120104966
Hi! Daddario, L2 distance: 0.3275074120104966


(0.3275074120104966, 'Daddario')